In [12]:
from Adafruit_IO import Client, Feed
from multiprocessing import Process
from i2c_sensor_classes import Ky015Sensor, Ky018Sensor
from aio_helper_classes import HabitatMonitorClient, HabitatMonitorServer
from multiprocessing import Process
import time
import threading
import asyncio

In [13]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
from pynq.lib import rgbled
base = BaseOverlay("base.bit")

# Client Code
## ToDo
- Integrate Threading + LED + Button code from Ryan
- Need to reace to thresholds and blink leds
- Should be able to receive new threshold values from cloud
- should Read AIO uname + key from a local file (dont push the file to git)
- launch all client threads in the client_process fucniton 

## Threads for Client

#### RGB Led indicates certain actions performed by the client or threshold values indicator. Here is the brief description
* Color Blue: (like bluetooth pairing) Solid Blue(LED4) : To indicate Threshold update. Blinking Blue: to indicate threshold out of date.
* Color Red(LED4): Blink Cloud parameter update failure.
* Color Red: (LED5)To indicate any parameter out of threshold limits. You could add different blink rates for temp, humidity, brightness.
* Color Green(LED5) To indicate all good and within threshold

In [14]:
def client_button(btns):
    global stop
    while not stop:
        if btns.read() == 0x01:
            stop = True
    print("Button was pressed! Ending program...")
    
def client_LED(LEDStates, LEDLock):
    #Process to handle blinking the LEDs to show various statuses on the client board
    blinkrate = 1
    t_on = 1/(2*blinkrate)
    
    blinkOn = False; #want lights to blink in sync, this stores state
    
    while not stop:
        usingLock = LEDLock.acquire(True)
        if usingLock:
            for i in range(4):
                if LEDStates[i] == 1:
                    if blinkOn:
                        base.leds[i].off()
                    else:
                        base.leds[i].on()

            for i in range(4,6):
                if LEDStates[i] == 1:
                    if blinkOn:
                        base.rgbleds[i].off()
                    else:
                        base.rgbleds[i].on(0b100)
            LEDLock.release()
        if blinkOn:
            blinkOn = False
        else:
            blinkOn = True
                 
        time.sleep(t_on)
    
def start_blink(LEDStates,index):
    LEDStates[index] = 1
    
def stop_blink(LEDStates,index):
    LEDStates[index] = 0
    led_off(LEDStates,index)
    
def blinkAllLEDs():
    for i in LEDStates:
        i = 1
    
def led_off(LEDStates,index):
    if index < 0 or index > 6:
        return
    elif(LEDStates[index] == 1):
        return
    elif index < 4:
        base.leds[index].off()
    else:
        base.rgbleds[index].off()
        
def led_on(LEDStates,index):
    if index < 0 or index > 6:
        return
    elif(LEDStates[index] == 1):
        return
    elif index < 4:
        base.leds[index].on()
    else:
        base.rgbleds[index].on(0b100)

In [15]:
def client_sensors(aio,LEDStates,LEDLock):
    asyncio.set_event_loop(asyncio.new_event_loop()) #required to avoid no event loop error
    
    KY015_ADDR = 8
    KY018_ADDR = 0x28
    SCL_PIN = 2
    SDA_PIN = 3

    liba = MicroblazeLibrary(base.PMODA, ['i2c'])
    #custom class see 'src/i2c_sensor_classes.py'
    ky015 = Ky015Sensor(liba, SDA_PIN, SCL_PIN, KY015_ADDR)

    libb = MicroblazeLibrary(base.PMODB, ['i2c'])
    #custom class see 'src/i2c_sensor_classes.py'
    ky018 = Ky018Sensor(libb, SDA_PIN, SCL_PIN, KY018_ADDR)

    lbtFeed = aio.feeds('low-brightness-threshold')
    hbtFeed = aio.feeds('high-brightness-threshold')
    lttFeed = aio.feeds('low-temperature-threshold')
    httFeed = aio.feeds('high-temperature-threshold')
    lhtFeed = aio.feeds('low-humidity-threshold')
    hhtFeed = aio.feeds('high-humidity-threshold')
    
    tempIsGood = True
    humIsGood = True
    thresholdsUpToDate = True
    samplectr = 0
    numberOfFailedPushes = 0
    while not stop:
        start_time = time.time()
        if samplectr == 0:
            #led_on(LEDStates,2) #solid LED 2 indicates updating thresholds
            try:
                [lowB, highB] = updateBrightnessThresholds(aio,lbtFeed,hbtFeed)
                [lowT, highT] = updateTemperatureThresholds(aio,lttFeed,httFeed)
                [lowH, highH] = updateHumidityThresholds(aio,lhtFeed,hhtFeed)
                if not thresholdsUpToDate:
                    thresholdsUpToDate = True
                    usinglock = LEDLock.acquire(True)
                    if usinglock:
                        stop_blink(LEDStates,4)
                        LEDLock.release()
            except:
                thresholdsUpToDate = False
                print("Error updating thresholds")
                start_blink(LEDStates,4) #blinking LED 4 (cloud error) to indicate threshold may be out of date
            #led_off(LEDStates,2)
        samplectr = (samplectr + 1) % SAMPLES_BEFORE_THRESHOLD_UPDATE
        
        temp, hum = ky015.get_temp_and_hum_data()
        bright = ky018.get_brightness_data()
        print('\tTemp: {}C, Hum: {}%\n\tBright: {}'.format(temp,hum, bright))
        
        #Temperature check
        if temp < lowT:
            tempIsGood = False
            start_blink(LEDStates,5)
            print("ALERT: Temperature is lower than threshold!")
        elif temp > highT:
            tempIsGood = False
            start_blink(LEDStates,5)
            print("ALERT: Temperature is higher than threshold!")
        else:
            tempIsGood = True
        
        #Humidity check
        if hum < lowH:
            humIsGood = False
            start_blink(LEDStates,5)
            print("ALERT: Humidity is lower than threshold!")
        elif hum > highH:
            humIsGood = False
            start_blink(LEDStates,5)
            print("ALERT: Humidity is higher than threshold!")
        else:
            humIsGood = True
        
        #Brightness check
        if bright < lowB:
            start_blink(LEDStates,5)
            print("ALERT: Brightness is lower than threshold!")
        elif bright > highB:
            start_blink(LEDStates,5)
            print("ALERT: Brightness is higher than threshold!")
        else:
            if tempIsGood and humIsGood:
                usinglock = LEDLock.acquire(True)
                if usinglock:
                    stop_blink(LEDStates,5)
                    LEDLock.release()
        
        
        try:
            aio.send(aio.temperature_feed.key, temp)
            aio.send(aio.humidity_feed.key, hum)
            aio.send(aio.brightness_feed.key, bright)
            print('\tData Sent to Adafruit.io')
            if numberOfFailedPushes > 0:
                numberOfFailedPushes = 0
                usinglock = LEDLock.acquire(True)
                if usinglock:
                    stop_blink(LEDStates,4)
                    LEDLock.release()
            #led_on(LEDStates,3)
            #time.sleep(0.3) #A blink on LED 3 indicates a successful push to the cloud
            #led_off(LEDStates,3)
        except:
            print("Error pushing data to adafruit.io")
            numberOfFailedPushes += 1
            if numberOfFailedPushes >= 5:
                start_blink(LEDStates,4)
        
        wake_time = start_time + SAMPLE_PERIOD
        sleep_time = wake_time - time.time()
        if (sleep_time > 0):
            time.sleep(sleep_time)
    
def updateBrightnessThresholds(aio,lbtFeed,hbtFeed):
    lowB = int(aio.receive(lbtFeed.key).value)
    highB = int(aio.receive(hbtFeed.key).value)
    print("Brightness threshold updated: {} to {}".format(lowB,highB))
    return [lowB,highB]

def updateTemperatureThresholds(aio,lttFeed,httFeed):
    lowT = float(aio.receive(lttFeed.key).value)
    highT = float(aio.receive(httFeed.key).value)
    print("Temperature threshold updated: {}C to {}C".format(lowT,highT))
    return [lowT,highT]

def updateHumidityThresholds(aio,lhtFeed,hhtFeed):
    lowH = float(aio.receive(lhtFeed.key).value)
    highH = float(aio.receive(hhtFeed.key).value)
    print("Humidity threshold updated: {}% to {}%".format(lowH,highH))
    return [lowH,highH]

In [16]:
def client_process(client_id, adakeypath):
    
    with open(adakeypath) as file:
        UNAME = file.readline().rstrip('\n')
        KEY = file.readline().rstrip('\n')
        
    client_id = client_id
    
    global stop
    global SAMPLE_PERIOD 
    SAMPLE_PERIOD = 60 #number of seconds to wait between each sample
    global SAMPLES_BEFORE_THRESHOLD_UPDATE
    SAMPLES_BEFORE_THRESHOLD_UPDATE = 2  #number of samples to take before updating threshold again

    #custom class see "src/aio_helper_classes.py"
    aio = HabitatMonitorClient(UNAME, KEY, client_id)

    stop = False

    LEDStates = [0, 0, 0, 0, 0, 0] #store states of LEDs. 0 = steady, 1 = blinking

    LEDLock = threading.Lock()

    btns = base.btns_gpio

    t1 = threading.Thread(target=client_sensors, args=(aio,LEDStates,LEDLock))
    t1.start()
    t2 = threading.Thread(target=client_LED, args=(LEDStates,LEDLock)) 
    t2.start()
    t3 = threading.Thread(target=client_button, args=(btns,))
    t3.start()
    t1.join()
    print('t1 joined')
    t2.join()
    print('t2 joined')
    t3.join()
    print('t3 joined')
    print('Client process exited.')

# Server Code
## ToDo
Create a server that can monitor the cloud feeds and change thresholds

In [17]:
class serverThresholdParam():
    def __init__(self, ht,lt,hh,lh,hb,lb):
        self.ht = ht
        self.lt = lt
        self.hh = hh
        self.lh = lh
        self.hb = hb
        self.lb = lb

In [18]:
server_leds = {
    0 : base.leds[0],
    1 : base.leds[1],
    2 : base.leds[2],
    3 : base.leds[3],
    4 : base.rgbleds[4],
}


In [19]:
def server_led_blink(blink_time, blink_rate, n):
    '''
    Function to blink the LEDs
    Params:
      blink_time: total time to blink led
      blink_rate: frequency to blink led
      num: the led id to blink
      c: optional argument to specify rbg color. defaults to green
    '''
    global server_leds
    #check for valid args
    t = round(blink_time/(1/blink_rate)) #converting the time into counter value
    for i in range(t):
        if 0 <= n and n< 4:
            server_leds[n].toggle()
            time.sleep(1/blink_rate)
        else:
            print("Invalid LED IDs to blink")
            return
    if 0 <= n and n < 4:
        server_leds[n].off()

def server_turn_off_leds(leds):
    '''
    Function to turn off all leds
    Params:
      leds: the leds to turn off
    '''
    for key in leds:
        leds[key].off()

In [20]:
ERROR_RATE = 10#hz
STABLE_RATE = 1 #
count = 0
Blink_Time = 2
while count < 5:
    server_led_blink(Blink_Time, STABLE_RATE,0)
    count +=1

In [21]:
def server_process(serverName, adafruitpath, serverConfig):
    #create an aio client 
    
    global stop
    global SAMPLE_PERIOD 
    SAMPLE_PERIOD = 60*2 #number of seconds to wait between each sample
    global SAMPLES_BEFORE_THRESHOLD_UPDATE
    SAMPLES_BEFORE_THRESHOLD_UPDATE = 60*5  #number of samples to take before updating threshold again

    #custom class see "src/aio_helper_classes.py"
#     client_id = "vinit-pynq-board"
    with open(adafruitpath) as file:
        uname = file.readline().rstrip('\n')
        key = file.readline().rstrip('\n')

    
    aio = HabitatMonitorServer(uname, key, serverName)
    #note these lines also push the new value to the could automatically
    aio.high_temp, aio.low_temp = serverConfig.ht, serverConfig.lt 
    aio.high_hum, aio.low_hum = serverConfig.hh, serverConfig.lh
    aio.high_bright, aio.low_bright = serverConfig.hb, serverConfig.lb
    
    while(base.btns_gpio.read() != 0b0110):
        monitor_thresholds(aio, server_leds)
        time.sleep(120)
    
    ##Create thread to monitor threshold
#     st1 = threading.Thread(target=monitor_thresholds, args=(aio,base.leds))
#     st1.start()
#     st1.join()
    
    
#     t1.join()
#     print('t1 joined')
#     t2.join()
#     print('t2 joined')
#     t3.join()
#     print('t3 joined')
#     print('Client process exited.')
    
#     print('st1 joined')
    
def monitor_thresholds(aio:HabitatMonitorServer, led):
    while True:
        monitor_temp_thresholds(aio,led)
        monitor_hum_thresholds(aio,led)
        monitor_bright_thresholds(aio, led)
    
def monitor_temp_thresholds(aio:HabitatMonitorServer, led):
    for tfeed in aio.temperature_group.feeds:
        print(tfeed.key)
        print(aio.get_client_num(tfeed.key))
        id_client = aio.get_client_num(tfeed.key)
        last_reading = float(aio.receive(tfeed.key).value)
        if last_reading > aio.high_temp:
            print('\tWarning: Temp={} > {}!'.format(last_reading, aio.high_temp))
            server_led_blink(Blink_Time, ERROR_RATE, id_client)
            aio.send(aio.client_status_feed.key, '{}, {}'.format(tfeed.key, aio.HIGHER_THEN_THRESHOLD))
        elif last_reading < aio.low_temp:
            print('\tWarning: Temp={} < {}!'.format(last_reading, aio.low_temp))
            server_led_blink(Blink_Time, ERROR_RATE, id_client)
            aio.send(aio.client_status_feed.key, '{}, {}'.format(tfeed.key, aio.LOWER_THEN_THRESHOLD))
        else:
            print('\tAll Good: {} < Temp={} < {}'.format(aio.low_temp, last_reading, aio.high_temp))
            server_led_blink(Blink_Time, STABLE_RATE, id_client)

def monitor_hum_thresholds(aio:HabitatMonitorServer, led):
    for hfeed in aio.humidity_group.feeds:
        print(hfeed.key)
        last_reading = float(aio.receive(hfeed.key).value)
        id_client = aio.get_client_num(hfeed.key)
        if last_reading > aio.high_hum:
            print('\tWarning: Hum={} > {}!'.format(last_reading, aio.high_hum))
            server_led_blink(Blink_Time, ERROR_RATE, id_client)
            aio.send(aio.client_status_feed.key, '{}, {}'.format(hfeed.key, aio.HIGHER_THEN_THRESHOLD))
        elif last_reading < aio.low_hum:
            print('\tWarning: Hum={} > {}!'.format(last_reading, aio.low_hum))
            server_led_blink(Blink_Time, ERROR_RATE, id_client)
            aio.send(aio.client_status_feed.key, '{}, {}'.format(hfeed.key, aio.LOWER_THEN_THRESHOLD))
        else:
            print('\tAll good {} < Hum={} < {}'.format(aio.low_hum, last_reading, aio.high_hum))
            server_led_blink(Blink_Time, STABLE_RATE, id_client)
        
def monitor_bright_thresholds(aio:HabitatMonitorServer, led):
    for bfeed in aio.brightness_group.feeds:
        print(bfeed.key)
        last_reading = float(aio.receive(bfeed.key).value)
        id_client = aio.get_client_num(bfeed.key)
        if last_reading > aio.high_bright:
            print('\tWarning: Bright={} > {}!'.format(last_reading, aio.high_bright))
            server_led_blink(Blink_Time, ERROR_RATE, id_client)
            aio.send(aio.client_status_feed.key, '{}, {}'.format(bfeed.key, aio.HIGHER_THEN_THRESHOLD))
        elif last_reading < aio.low_bright:
            print('\tWarning: Bright={} < {}!'.format(last_reading, aio.low_bright))
            server_led_blink(Blink_Time, ERROR_RATE, id_client)
            aio.send(aio.client_status_feed.key, '{}, {}'.format(bfeed.key, aio.LOWER_THEN_THRESHOLD))
        else:
            print('\tAll good {} < Bright={} < {}'.format(aio.low_bright, last_reading, aio.high_bright))
            server_led_blink(Blink_Time, STABLE_RATE, id_client)

### Main Server function
* If server does not run other client overwrites their own threshold
* If server runs, server overwrites threshold.

In [22]:
    high_temp   = 100#   = float(input("Enter Higher temperature threshold "))
    low_temp    = 50 #= float(input("Enter Lower temperature threshold "))
    high_hum    = 90 #= float(input("Enter Higher humidity threshold "))
    low_hum     = 25 #= float(input ("Enter lower humidity threshold "))
    high_bright =1000#= float(input("Enter Higher threshold bright light "))
    low_bright  = 800#= float(input("Enter lower threshold bright light "))
    
    serverConfig = serverThresholdParam(high_temp, low_temp, high_hum, low_hum, high_bright, low_bright)
    

    #server_process('server_1', 'adafruitkey.dontpush', serverConfig)

# main() Function
## ToDo
- Launch client process 
- Optionally launch server process

In [32]:
def main():
    client_id = 'joeys-pynq-board'
    server_id = 'joeys-pynq-server'
    adakeypath = 'adafruitkey.dontpush'
    high_temp   = 50
    low_temp    = 15 
    high_hum    = 90 
    low_hum     = 25
    high_bright =3000
    low_bright  = 800
    server_config = serverThresholdParam(high_temp, low_temp, high_hum, low_hum, high_bright, low_bright)
    p1 = Process(target=client_process, args=(client_id,adakeypath)) 
    p1.start() # start the process
    p2 = Process(target=server_process, args=(server_id,adakeypath, server_config))
    p2.start()
    p1.join()    
    print('p1 joined')
    p2.join()
    print('p2 joined')
    print('Program Done!')

In [ ]:
main()

creating new HabitatMonitor...
Creating Habitat Mon Server...
HabitatMonitorClientsetup
temperature-measurements.temperature-joeys-pynq-board-1
1
	All Good: 15 < Temp=20.08 < 50
temperature-measurements.temperature-ryans-pynq-board-2
2
	All Good: 15 < Temp=22.04 < 50
temperature-measurements.temperature-vinit-pynq-board-3
3
	All Good: 15 < Temp=22.04 < 50
humidity-measurements.humidity-joeys-pynq-board-1
Brightness threshold updated: 800 to 3000
	All good 25 < Hum=46.0 < 90
Temperature threshold updated: 15.0C to 50.0C
Humidity threshold updated: 25.0% to 90.0%
humidity-measurements.humidity-ryans-pynq-board-2
	Temp: 20.08C, Hum: 46.0%
	Bright: 1624
	All good 25 < Hum=40.0 < 90
	Data Sent to Adafruit.io
humidity-measurements.humidity-vinit-pynq-board-3
	All good 25 < Hum=38.0 < 90
brightness-measurements.brightness-joeys-pynq-board-1
	All good 800 < Bright=1624.0 < 3000
brightness-measurements.brightness-ryans-pynq-board-2
	All good 800 < Bright=2689.0 < 3000
brightness-measurements.br

brightness-measurements.brightness-ryans-pynq-board-2
	All good 800 < Bright=2709.0 < 3000
brightness-measurements.brightness-vinit-pynq-board-3
temperature-measurements.temperature-joeys-pynq-board-1
1
	All Good: 15 < Temp=20.08 < 50
temperature-measurements.temperature-ryans-pynq-board-2
2
	All Good: 15 < Temp=22.04 < 50
temperature-measurements.temperature-vinit-pynq-board-3
3
	All Good: 15 < Temp=22.04 < 50
humidity-measurements.humidity-joeys-pynq-board-1
	All good 25 < Hum=46.0 < 90
Brightness threshold updated: 800 to 3000
Temperature threshold updated: 15.0C to 50.0C
humidity-measurements.humidity-ryans-pynq-board-2
	All good 25 < Hum=40.0 < 90
Humidity threshold updated: 25.0% to 90.0%
	Temp: 20.08C, Hum: 46.0%
	Bright: 1650
	Data Sent to Adafruit.io
humidity-measurements.humidity-vinit-pynq-board-3
	All good 25 < Hum=38.0 < 90
brightness-measurements.brightness-joeys-pynq-board-1
	All good 800 < Bright=1650.0 < 3000
brightness-measurements.brightness-ryans-pynq-board-2
	All g

brightness-measurements.brightness-joeys-pynq-board-1
	All good 800 < Bright=1580.0 < 3000
brightness-measurements.brightness-ryans-pynq-board-2
	All good 800 < Bright=2738.0 < 3000
brightness-measurements.brightness-vinit-pynq-board-3
temperature-measurements.temperature-joeys-pynq-board-1
1
	All Good: 15 < Temp=20.08 < 50
temperature-measurements.temperature-ryans-pynq-board-2
2
	All Good: 15 < Temp=22.04 < 50
temperature-measurements.temperature-vinit-pynq-board-3
3
	All Good: 15 < Temp=22.04 < 50
humidity-measurements.humidity-joeys-pynq-board-1
	All good 25 < Hum=46.0 < 90
Brightness threshold updated: 800 to 3000
humidity-measurements.humidity-ryans-pynq-board-2
Temperature threshold updated: 15.0C to 50.0C
	All good 25 < Hum=40.0 < 90
Humidity threshold updated: 25.0% to 90.0%
	Temp: 20.08C, Hum: 46.0%
	Bright: 1598
humidity-measurements.humidity-vinit-pynq-board-3
	All good 25 < Hum=38.0 < 90
	Data Sent to Adafruit.io
brightness-measurements.brightness-joeys-pynq-board-1
	All g

humidity-measurements.humidity-vinit-pynq-board-3
	All good 25 < Hum=38.0 < 90
brightness-measurements.brightness-joeys-pynq-board-1
	All good 800 < Bright=1403.0 < 3000
brightness-measurements.brightness-ryans-pynq-board-2
	All good 800 < Bright=2722.0 < 3000
brightness-measurements.brightness-vinit-pynq-board-3
temperature-measurements.temperature-joeys-pynq-board-1
1
	All Good: 15 < Temp=20.06 < 50
temperature-measurements.temperature-ryans-pynq-board-2
2
	All Good: 15 < Temp=22.04 < 50
temperature-measurements.temperature-vinit-pynq-board-3
3
	All Good: 15 < Temp=22.02 < 50
humidity-measurements.humidity-joeys-pynq-board-1
	All good 25 < Hum=46.0 < 90
Brightness threshold updated: 800 to 3000
humidity-measurements.humidity-ryans-pynq-board-2
	All good 25 < Hum=40.0 < 90
Temperature threshold updated: 15.0C to 50.0C
Humidity threshold updated: 25.0% to 90.0%
	Temp: 20.06C, Hum: 46.0%
	Bright: 1423
humidity-measurements.humidity-vinit-pynq-board-3
	All good 25 < Hum=38.0 < 90
	Data S

humidity-measurements.humidity-ryans-pynq-board-2
	All good 25 < Hum=40.0 < 90
humidity-measurements.humidity-vinit-pynq-board-3
	All good 25 < Hum=38.0 < 90
brightness-measurements.brightness-joeys-pynq-board-1
	All good 800 < Bright=1651.0 < 3000
brightness-measurements.brightness-ryans-pynq-board-2
	All good 800 < Bright=2690.0 < 3000
brightness-measurements.brightness-vinit-pynq-board-3
temperature-measurements.temperature-joeys-pynq-board-1
1
	All Good: 15 < Temp=20.08 < 50
temperature-measurements.temperature-ryans-pynq-board-2
2
	All Good: 15 < Temp=22.04 < 50
temperature-measurements.temperature-vinit-pynq-board-3
3
	All Good: 15 < Temp=22.02 < 50
humidity-measurements.humidity-joeys-pynq-board-1
	All good 25 < Hum=46.0 < 90
humidity-measurements.humidity-ryans-pynq-board-2
	All good 25 < Hum=40.0 < 90
Brightness threshold updated: 800 to 3000
Temperature threshold updated: 15.0C to 50.0C
humidity-measurements.humidity-vinit-pynq-board-3
	All good 25 < Hum=38.0 < 90
Humidity th

humidity-measurements.humidity-joeys-pynq-board-1
	All good 25 < Hum=46.0 < 90
humidity-measurements.humidity-ryans-pynq-board-2
	All good 25 < Hum=40.0 < 90
humidity-measurements.humidity-vinit-pynq-board-3
	All good 25 < Hum=38.0 < 90
brightness-measurements.brightness-joeys-pynq-board-1
	All good 800 < Bright=1642.0 < 3000
brightness-measurements.brightness-ryans-pynq-board-2
	All good 800 < Bright=2740.0 < 3000
brightness-measurements.brightness-vinit-pynq-board-3
temperature-measurements.temperature-joeys-pynq-board-1
1
	All Good: 15 < Temp=20.06 < 50
temperature-measurements.temperature-ryans-pynq-board-2
2
	All Good: 15 < Temp=22.04 < 50
temperature-measurements.temperature-vinit-pynq-board-3
3
	All Good: 15 < Temp=22.02 < 50
humidity-measurements.humidity-joeys-pynq-board-1
	All good 25 < Hum=46.0 < 90
humidity-measurements.humidity-ryans-pynq-board-2
	All good 25 < Hum=40.0 < 90
Brightness threshold updated: 800 to 3000
humidity-measurements.humidity-vinit-pynq-board-3
	All go

temperature-measurements.temperature-vinit-pynq-board-3
3
	All Good: 15 < Temp=22.02 < 50
humidity-measurements.humidity-joeys-pynq-board-1
	All good 25 < Hum=46.0 < 90
humidity-measurements.humidity-ryans-pynq-board-2
	All good 25 < Hum=40.0 < 90
humidity-measurements.humidity-vinit-pynq-board-3
	All good 25 < Hum=38.0 < 90
brightness-measurements.brightness-joeys-pynq-board-1
	All good 800 < Bright=1675.0 < 3000
brightness-measurements.brightness-ryans-pynq-board-2
	All good 800 < Bright=2716.0 < 3000
brightness-measurements.brightness-vinit-pynq-board-3
temperature-measurements.temperature-joeys-pynq-board-1
1
	All Good: 15 < Temp=20.06 < 50
temperature-measurements.temperature-ryans-pynq-board-2
2
	All Good: 15 < Temp=22.02 < 50
temperature-measurements.temperature-vinit-pynq-board-3
3
	All Good: 15 < Temp=22.02 < 50
humidity-measurements.humidity-joeys-pynq-board-1
	All good 25 < Hum=46.0 < 90
humidity-measurements.humidity-ryans-pynq-board-2
	All good 25 < Hum=40.0 < 90
humidity-

temperature-measurements.temperature-ryans-pynq-board-2
2
	All Good: 15 < Temp=22.02 < 50
temperature-measurements.temperature-vinit-pynq-board-3
3
	All Good: 15 < Temp=22.02 < 50
humidity-measurements.humidity-joeys-pynq-board-1
	All good 25 < Hum=46.0 < 90
humidity-measurements.humidity-ryans-pynq-board-2
	All good 25 < Hum=40.0 < 90
humidity-measurements.humidity-vinit-pynq-board-3
	All good 25 < Hum=38.0 < 90
brightness-measurements.brightness-joeys-pynq-board-1
	All good 800 < Bright=1382.0 < 3000
brightness-measurements.brightness-ryans-pynq-board-2
	All good 800 < Bright=2737.0 < 3000
brightness-measurements.brightness-vinit-pynq-board-3
temperature-measurements.temperature-joeys-pynq-board-1
1
	All Good: 15 < Temp=20.06 < 50
temperature-measurements.temperature-ryans-pynq-board-2
2
	All Good: 15 < Temp=22.02 < 50
temperature-measurements.temperature-vinit-pynq-board-3
3
	All Good: 15 < Temp=22.02 < 50
humidity-measurements.humidity-joeys-pynq-board-1
	All good 25 < Hum=46.0 < 9

temperature-measurements.temperature-joeys-pynq-board-1
1
	All Good: 15 < Temp=20.06 < 50
temperature-measurements.temperature-ryans-pynq-board-2
2
	All Good: 15 < Temp=22.04 < 50
temperature-measurements.temperature-vinit-pynq-board-3
3
	All Good: 15 < Temp=22.02 < 50
humidity-measurements.humidity-joeys-pynq-board-1
	All good 25 < Hum=46.0 < 90
humidity-measurements.humidity-ryans-pynq-board-2
	All good 25 < Hum=40.0 < 90
humidity-measurements.humidity-vinit-pynq-board-3
	All good 25 < Hum=38.0 < 90
brightness-measurements.brightness-joeys-pynq-board-1
	All good 800 < Bright=1677.0 < 3000
brightness-measurements.brightness-ryans-pynq-board-2
	All good 800 < Bright=2487.0 < 3000
brightness-measurements.brightness-vinit-pynq-board-3
temperature-measurements.temperature-joeys-pynq-board-1
1
	All Good: 15 < Temp=20.06 < 50
temperature-measurements.temperature-ryans-pynq-board-2
2
	All Good: 15 < Temp=22.04 < 50
temperature-measurements.temperature-vinit-pynq-board-3
3
	All Good: 15 < Tem

brightness-measurements.brightness-vinit-pynq-board-3
temperature-measurements.temperature-joeys-pynq-board-1
1
	All Good: 15 < Temp=20.06 < 50
temperature-measurements.temperature-ryans-pynq-board-2
2
	All Good: 15 < Temp=22.04 < 50
temperature-measurements.temperature-vinit-pynq-board-3
3
	All Good: 15 < Temp=22.02 < 50
humidity-measurements.humidity-joeys-pynq-board-1
	All good 25 < Hum=46.0 < 90
humidity-measurements.humidity-ryans-pynq-board-2
	All good 25 < Hum=40.0 < 90
humidity-measurements.humidity-vinit-pynq-board-3
	All good 25 < Hum=38.0 < 90
brightness-measurements.brightness-joeys-pynq-board-1
	All good 800 < Bright=1663.0 < 3000
brightness-measurements.brightness-ryans-pynq-board-2
	All good 800 < Bright=2716.0 < 3000
brightness-measurements.brightness-vinit-pynq-board-3
temperature-measurements.temperature-joeys-pynq-board-1
1
	All Good: 15 < Temp=20.06 < 50
temperature-measurements.temperature-ryans-pynq-board-2
2
	All Good: 15 < Temp=22.04 < 50
temperature-measurement

In [30]:
client_process('joeys-pynq-board', 'adafruitkey.dontpush')

creating new HabitatMonitor...
HabitatMonitorClientsetup
Brightness threshold updated: 800 to 3555
Temperature threshold updated: 10.0C to 100.0C
Humidity threshold updated: 25.0% to 90.0%
	Temp: 20.06C, Hum: 46.0%
	Bright: 1738
	Data Sent to Adafruit.io
	Temp: 20.06C, Hum: 46.0%
	Bright: 1740
	Data Sent to Adafruit.io
Button was pressed! Ending program...
t1 joined
t2 joined
t3 joined
Client process exited.
